## `Oligopool Calculator` in Action!

**Author** Ayaan Hossain

**Updated** January 25, 2026

This `jupyter` notebook will demonstrate the usage of `Oligopool Calculator` and describe its many functions.

### Content

* [Notebook Setup](#Notebook-Setup)
* [What is Oligopool Calculator?](#What-is-Oligopool-Calculator?)
* [Design Mode Walkthrough](#Design-Mode-Walkthrough)
    * [Simulating the problem](#Simulating-the-problem)
    * [Reviewing constraints and formulating gameplan](#Reviewing-constraints-and-formulating-gameplan)
    * [Primer design review](#Primer-design-review)
    * [Storing the plasmid background](#Storing-the-plasmid-background)
    * [Designing the triple-primer system](#Designing-the-triple-primer-system)
    * [Generating orthogonal barcodes](#Generating-orthogonal-barcodes)
    * [Adding neutral motifs, spacers, and finalizing library](#Adding-neutral-motifs,-spacers,-and-finalizing-library)
* [Assembly Mode Walkthrough](#Assembly-Mode-Walkthrough)
    * [The Challenge: Synthesis Length Limits](#The-Challenge:-Synthesis-Length-Limits)
    * [Splitting long oligos into overlapping fragments](#Splitting-long-oligos-into-overlapping-fragments)
    * [Padding fragments for Type IIS assembly](#Padding-fragments-for-Type-IIS-assembly)
* [Degenerate Mode Walkthrough](#Degenerate-Mode-Walkthrough)
    * [The Challenge: Expensive Similar-Sequence Libraries](#The-Challenge:-Expensive-Similar-Sequence-Libraries)
    * [Analysis Mode vs Degenerate Mode](#Analysis-Mode-vs-Degenerate-Mode)
    * [The Degenerate Mode Pipeline](#The-Degenerate-Mode-Pipeline)
    * [Understanding IUPAC Degenerate Codes](#Understanding-IUPAC-Degenerate-Codes)
    * [Compressing a library into degenerate oligos](#Compressing-a-library-into-degenerate-oligos)
    * [Realistic Example: 500 ML-style variants](#Realistic-Example:-500-ML-style-variants)
    * [Expanding degenerate oligos](#Expanding-degenerate-oligos)
* [Analysis Mode Walkthrough](#Analysis-Mode-Walkthrough)
    * [The Challenge: Extracting Signal from Sequencing Noise](#The-Challenge:-Extracting-Signal-from-Sequencing-Noise)
    * [Indexing the barcodes and associated variants](#Indexing-the-barcodes-and-associated-variants)
    * [Packing the NGS reads for quantification](#Packing-the-NGS-reads-for-quantification)
    * [Overview of counting methods](#Overview-of-counting-methods)
    * [`acount` vs `xcount`: Choosing Your Counting Method](#acount-vs-xcount:-Choosing-Your-Counting-Method)
    * [Association counting of variants and barcodes](#Association-counting-of-variants-and-barcodes)
    * [Combinatorial barcode counting with callback](#Combinatorial-barcode-counting-with-callback)
* [Conclusion](#Conclusion)

### Notebook Setup

We will need to have `Oligopool Calculator` installed to proceed. You can install it by uncommenting and executing the following cell.

In [ ]:
# !pip install oligopool

Let's import `oligopool`.

In [ ]:
import oligopool as op

We will need `numpy` to do some math, and we will use `pandas` as our data container.

In [ ]:
import numpy  as np
import pandas as pd

We will also need `matplotlib`/`seaborn` for plotting, and `pprint` for pretty printing.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

Finally, we will need `multiprocessing` `Manager` to create a shared dictionary for demonstrating synchronized callback memory during counting.

In [ ]:
from multiprocessing import Manager

We're all set.

### What is Oligopool Calculator?

Oligonucleotide pools (oligo pools) have revolutionized synthetic biology by enabling large-scale, cost-effective synthesis of hundreds of thousands of unique, short single-stranded DNA sequences.

These oligo pools facilitate the creation and study of various biological components, including de novo promoters, ribozymes, protein scaffolds, and CRISPR libraries. Massively parallel reporter assays (MPRAs) are utilized to clone and assess the functionality of these variants using high-throughput next-generation sequencing (NGS), which can generate billions of short reads per experiment.

Looking ahead, advancements are expected in the number and length of oligos, NGS throughput, and reductions in manufacturing costs.

![Oligopool Calculator Workflow](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/workflow.svg)

`Oligopool Calculator` is a comprehensive tool for designing and analyzing large-scale oligonucleotide pools.

In **(a)** `Design Mode`, you build an oligo architecture by adding constrained elements (primers, barcodes, motifs/anchors, spacers) and running QC as you go (`lenstat`, `verify`). Many element modules support **Patch Mode** (`patch_mode=True`) to fill only missing values when you append new rows to an already-designed pool.

In `Assembly Mode`, you handle libraries longer than the synthesis limit by fragmenting and padding them for downstream assembly (`split`, `pad`).

In `Degenerate Mode`, you can compress a library of similar concrete sequences into IUPAC-degenerate oligos for cost-efficient synthesis (`compress`), and optionally expand them back as a correctness check (`expand`). Best for selection assays where you identify winners by sequencing (no barcode counting required).

If you prefer CLI-first workflows, the CLI supports YAML config files for both single commands and multi-step pipelines (see `examples/cli-pipeline/`).

In **(b)** `Analysis Mode`, `Oligopool Calculator` facilitates the processing of next-generation sequencing (NGS) data derived from barcoded amplicons. This mode allows for efficient mapping of variants and their associated barcodes, producing count matrices for quantitative assessment of variant functionality. Two types of counting operations are supported: Association Counting for assessing synthesis accuracy, and Combinatorial Counting for analyzing multiple barcode combinations from sequencing reads.

Overall `Oligopool Calculator` provides synthetic biologists with a unified platform for oligo pool design and characterization, addressing the need for a standardized approach across different projects and labs. This tool eliminates the need for reinventing design scripts or using suboptimal strategies, making it particularly valuable for designing and analyzing pools containing millions of defined variants.

### Design Mode Walkthrough

Suppose we want to design and test a library of 6,232 ribozyme variants based on the following architecture.

![Ribozyme Library Architecture](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/architecture-ribozyme.svg)

To characterize the activity of our library in _E. coli_, we adopt a two-barcode strategy. Because ribozymes are self-cleaving, the two barcodes (BC1 and BC2), placed before and after the core variant, are separated after catalysis. This lets us quantify cleavage by counting the ratio of 'separated' to 'unseparated' reads.

When using multiple barcode columns, it can also be useful to enforce *cross-set* separation (e.g., ensure BC2 barcodes are also far from the BC1 barcode set), which we will demonstrate later.

#### Simulating the problem

For the purposes of demonstration, let us first generate a random library of 59-73 mers (designing exact ribozymes is beyond the scope here) and assume that the library will be cloned into a 5 kb plasmid backbone for characterization, which we will also generate randomly.

In [ ]:
rng = np.random.default_rng(seed=42)

In [ ]:
dna = ['A', 'T', 'G', 'C']

In [ ]:
ribozymes = []
while len(ribozymes) < 6232:
    variant = ''.join(rng.choice(dna, size=rng.integers(low=59, high=74)))
    if 'GAATTC' in variant or 'GACGTC' in variant or 'GGTCTC' in variant or 'GAGACC' in variant:
        # Skipping if random variant has EcoRI, AatII, or BsaI motif
        continue
    ribozymes.append(variant)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(20,5))
sns.histplot([len(x) for x in ribozymes], binwidth=1)
ax.set_xticks(np.arange(59,74))
ax.set_xlabel('Ribozyme Length')
plt.show()

In [ ]:
len(ribozymes)

In [ ]:
plasmid = ''.join(rng.choice(dna) for _ in range(5000))

In [ ]:
len(plasmid) # 5 kbp long plasmid

#### Reviewing constraints and formulating gameplan

We have generated 6,232 simulated candidates between 59 to 73 bp in length, and a plasmid of length 5 kbp. We want to use `EcoRI` and `AatII` as our restriction sites as part of the characterization process.

We can now use `Oligopool Calculator` to design the following.
* Three primers, each 20bp in length,
* The prefix and suffix barcodes each 11bp in length, and
* Variable length spacers ranging from 3 to 17bp to pad the oligos to 170bp.

As constraints, we aim to have the following.
* The primers should not have off-target binding within the library or to the backbone,
* Addition of primers, barcodes or spacers should not contain `EcoRI`, `AatII`, or `BsaI` (Type IIS for padding) motifs or introduce them at the edges, and
* The barcodes should all have at least 3 mismatches separating each other.

Other realistic constraints, such as exclusion of all palindromic hexamers, or polymeric runs such as `'AAAAA'`, ..., `'CCCCC'` may be included as they were in the original projects, but we will skip those for this demonstration. In reality, constraints will depend on the design project.

A useful habit in Design Mode is to measure and QC as we go:

- `lenstat` is a fast length ruler: it reports per-element and per-oligo lengths and remaining free space under an `oligo_length_limit` (it does not modify the input DataFrame).
- `verify` is a broader QC pass (often right before synthesis): it summarizes sequence vs metadata columns, flags degenerate/mixed columns, and checks length overflow + excluded motif emergence.

How do we begin? By asking `help` from `Oligopool Calculator`.

In [ ]:
help(op)

Per the help, we will need to primarily use the `barcode`, `primer`, and `spacer` modules to add the library elements.

These element modules (`barcode`, `primer`, `motif`, `spacer`) also support **Patch Mode** (`patch_mode=True`), which is designed for iterative workflows:

- If you **append new rows** to an already-designed pool (e.g., you already ordered BC1/primers, then later add new variants), Patch Mode fills only the missing values in an existing output column and leaves existing designs untouched.
- This makes **vertical expansion** safe: you can extend a pool without redesigning (and potentially changing) existing barcodes/primers/anchors/spacers.

In Patch Mode, missing values (e.g., `None`/NaN/empty/`'-'`) are treated as design targets; existing values are sanity-checked (DNA + expected length) but are not re-designed.

#### Primer design review

Generally, primers are the most constrained elements in an oligo pool library because they have to follow a desired sequence constraint (such as `5'-GC` clamps), be free of inhibitory structures, be very specific to the library and have minimal off-targeting to background. We recommend designing primers as early as possible, but after all other elements in the library have been finalized, such as any motifs (discussed later in this notebook).

For multiplexed libraries, you can assign per-oligo set labels via `oligo_sets` to design primers independently for each set while ensuring cross-set compatibility. If a paired primer column is supplied, the paired primer is interpreted per set so that each group remains internally matched.


![Primer Design](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/primer.svg)

At any step in the process if we get stuck, `Oligopool Calculator` will tell us what is preventing it from succeeding and we can relax or modify our constraints accordingly.

How do we design primers?

In [ ]:
? op.primer

Two things should stand out here.

First, primer design involves consideration of background sequences to minimize off-target amplification, such as from the plasmid we set up. The same background can also be reused by other Design Mode modules (`barcode`, `motif`, `spacer`) and by `verify` via `background_directory`.

Second, primer design involves consideration of pairing information. If we only had two primers (one forward and one reverse) the order of their design would not matter. But in our problem, there are two forward primers (pink and orange) both of which are coupled to a single reverse primer (yellow). It will be important to design them in the right order, as we will discuss shortly.

Let us first solve the background problem.

#### Storing the plasmid background

`Oligopool Calculator` makes it easy to define a reusable background k-mer database via the `background` module.

In [ ]:
? op.background

> **Note:** If you want to modify the background post-hoc, or want to define your background using a custom strategy, directly use the `vectorDB` module, which behaves like a dictionary. For details use `help(op.vectorDB)`. The `background` module just wraps around it and offers an one-time use simple interface to `vectorDB`.

In [ ]:
background_directory = 'demo.oligopool.background' # Path to store our background

Let's remove the background if one exists already, for this demonstration.

In [ ]:
! rm -rf $background_directory

Execute the `background` module.

In [ ]:
background_stats = op.background(
    input_data=plasmid,
    maximum_repeat_length=8,
    output_directory=background_directory)

In [ ]:
pprint(background_stats)

We've now stored the background, and are ready to design the three primers (and any other elements you want to screen against the same background).

##### Multiple background DBs (optional)

All Design Mode modules that accept `background_directory` also accept multiple backgrounds via a list. If you pass multiple background DBs, designed elements are screened against ALL of them.

In [ ]:
# Example (not executed here):
background_directories = [
    'demo.oligopool.background',
    'ecoli_bg',  # resolves to ecoli_bg.oligopool.background
]

# df, stats = op.primer(..., background_directory=background_directories)
# df, stats = op.barcode(..., background_directory=background_directories)
# df, stats = op.motif(..., background_directory=background_directories)
# df, stats = op.spacer(..., background_directory=background_directories)
# stats = op.verify(..., background_directory=background_directories)


> **Note**: All `Oligopool Calculator` functions should return a pipeline statistics dictionary recapitulating the output from the last step performed in the function. If the operation completed successfully, the `status` key will be marked as `True` and `basis` will be marked as `'solved'`.

#### Designing the triple-primer system

All `Design Mode` input and output is mediated through a `pandas` `DataFrame` to facilitate carefully piplined designs. Let us store the variant candidates in a dataframe and start the design process.

In [ ]:
df = pd.DataFrame({
    'ID': [f'ribozyme_{x+1:04d}' for x in range(len(ribozymes))],
    'EcoRI': 'GAATTC',
    'Variant':ribozymes,
    'AatII': 'GACGTC',
})

In [ ]:
df.tail(5)

Given that the primers are coupled, an optimal solution here is to design the innermost (pink) forward primer first, and then design the reverse primer (yellow), followed by designing the outermost (orange) forward primer. This allows all three primers to be compatible with each other.

An alternate solution would involve designing the (yellow) reverse primer first, and then designing the two forward primers successively.

However, if we designed the (yellow) reverse primer at the end, it would be difficult to ensure that it is compatible with both forward primers simultaneously.

In [ ]:
# ? op.primer

In [ ]:
df, stats = op.primer(
    input_data=df,
    oligo_length_limit=170,                           # Must not exceed 170bp at any stage
    primer_sequence_constraint='SS' + 'N'*18,         # Manually install 5p GC clamp if required
    primer_type='forward',                            # Forward primer design
    primer_column='PinkForwardPrimer',                # Store designs in 'PinkForwardPrimer' column
    left_context_column='Variant',                    # Next to 'Variant' column
    right_context_column='AatII',                     # Before 'AatII' column
    paired_primer_column=None,                        # No paired primer yet
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'],  # Exclude EcoRI, AatII, BsaI motifs
    minimum_melting_temperature=53,                   # Tm = 53C at least
    maximum_melting_temperature=55,                   # Tm = 55C at most
    maximum_repeat_length=10,                         # No more than 10-mer repeats allowed
    background_directory=background_directory,        # As setup before
    output_file=None)                                 # In-memory return, no CSV written here

The designed primer is predicted to be free from hairpin structures with low homodimer probability.

Let us now design the two remaining primers in silent mode.

In [ ]:
df, _ = op.primer(
    input_data=df,                                   # Updated DataFrame along pipeline chain
    oligo_length_limit=170,
    primer_sequence_constraint='N'*18 + 'WW',        # Manually install 3p AT clamp if needed
    primer_type='reverse',                           # Reverse primer design
    primer_column='YellowReversePrimer',             # Store designs in 'YellowReversePrimer' column
    left_context_column='PinkForwardPrimer',         # Next to 'PinkForwardPrimer' column
    right_context_column='AatII',                    # Before 'AatII' column
    paired_primer_column='PinkForwardPrimer',        # Yellow reverse primer paired with pink forward primer
    minimum_melting_temperature=53,
    maximum_melting_temperature=55,
    maximum_repeat_length=10,
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'],  # Exclude EcoRI, AatII, BsaI motifs
    background_directory=background_directory,
    verbose=False)                               # Silent Mode

In [ ]:
df, _ = op.primer(
    input_data=df,                                    # Updated DataFrame along pipeline chain
    oligo_length_limit=170,
    primer_sequence_constraint='N'*20,
    primer_type='forward',                            # Forward primer design
    primer_column='OrangeForwardPrimer',              # Store designs in 'OrangeForwardPrimer' column
    left_context_column='EcoRI',                      # Next to 'EcoRI' column
    right_context_column='Variant',                   # Before 'Variant' column
    paired_primer_column='YellowReversePrimer',       # Orange forward primer paired with yellow reverse primer
    minimum_melting_temperature=53,
    maximum_melting_temperature=55,
    maximum_repeat_length=10,
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'],  # Exclude EcoRI, AatII, BsaI motifs
    background_directory=background_directory,
    verbose=False)

In [ ]:
df.head(5)

All our primers are installed. Barcodes up next!

Before that, since we're done with the background for this demo, let's drop it. In real projects you typically keep the background DB around and reuse it across multiple design steps.

In [ ]:
# help(op.vectorDB)

In [ ]:
op.vectorDB(background_directory, maximum_repeat_length=None).drop()

#### Generating orthogonal barcodes

Barcodes are critical elements that help identify each variant uniquely, especially when the core oligo variants are similar to each other. `Oligopool Calculator` maximizes barcode distinguishability which is useful, for example, when the library is exploring all single substitutions to a template.

![Barcode Design](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/barcode.svg)

In [ ]:
? op.barcode

Unlike primers which need to satisfy global constraints (a hard problem), barcodes are optimized contextually. So, barcodes may be designed after all the primers have been installed. However, the pairwise Hamming distance requirement coupled with other constraints makes it difficult to judge ahead of time if there are enough barcodes possible for a given library with desired constraints.

Let us design BC1 first.

In [ ]:
df.columns

In [ ]:
df, _ = op.barcode(
    input_data=df,
    oligo_length_limit=170,
    barcode_length=11,
    minimum_hamming_distance=3,  # As defined by our goal
    maximum_repeat_length=6,     # To prevent internal hairpins, say
    barcode_column='BC1',
    barcode_type='spectrum',     # Spectrum optimized barcodes
    left_context_column='OrangeForwardPrimer',
    right_context_column='Variant',
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'])  # Exclude EcoRI, AatII, BsaI motifs

Most of the barcodes have at least 4 mismatches separating them, and all of them are at least 3 mismatches apart as intended.

Next, let's add BC2.

**Cross-barcode sets (optional):** When designing multiple barcode columns (BC1, BC2, ...), you may want each new barcode set to be separated not only within itself (`minimum_hamming_distance`), but also from existing barcode sets.

`cross_barcode_columns` + `minimum_cross_distance` enforce a *global* cross-set Hamming distance between each newly designed barcode and the union of barcodes in the specified column(s) (not a per-row pairing constraint).

Below, BC2 is designed to be >=3 mismatches away from every BC1 sequence. For a third barcode, you could use `cross_barcode_columns=['BC1','BC2']`.

In [ ]:
df, _ = op.barcode(
    input_data=df,
    oligo_length_limit=170,
    barcode_length=11,
    minimum_hamming_distance=3,
    maximum_repeat_length=6,
    barcode_column='BC2',
    barcode_type='spectrum',     # Spectrum optimized barcodes
    left_context_column='PinkForwardPrimer',
    right_context_column='YellowReversePrimer',
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'],  # Exclude EcoRI, AatII, BsaI motifs
    cross_barcode_columns=['BC1'],
    minimum_cross_distance=3,
    verbose=False)  # Silent mode

In [ ]:
df.head(5)

Barcodes are done! The only thing left to design are the spacers so that all oligos have the same length of 170 bp.

Before designing spacers, we first measure how much length budget is left. The `lenstat` module is a quick ruler: it reports per-element lengths and remaining free space, without modifying the DataFrame.

In [ ]:
# help(op.lenstat)

In [ ]:
length_stats = op.lenstat(input_data=df, oligo_length_limit=170)

#### Adding neutral motifs, spacers, and finalizing library

In [ ]:
df.columns

The last element in our oligo is an `AatII` restriction site, and we want to add 3 to 17 bases of filler next to it. We can do this using the `spacer` module.

But suppose we wanted to add a fixed length spacer site based on a degenerate sequence constraint, such as a motif? We would use the `motif` module instead. One way to think about this is that spacers are a more general form of motifs -- motifs are more constrained in terms of sequence composition and are of fixed length.

Another important use of the `motif` module is in designing constant barcode anchors that is distinct from the remainder of the oligos (controlled via `maximum_repeat_length` parameter) to facilate downstream counting.

In [ ]:
? op.motif

We will see this function in action by first removing the `AatII` motif column and replacing it with a degenerate motif containing the constant restriction site embedded in it but with 3 `'N'`s on its right flank.

In [ ]:
df = df.drop(columns=['AatII'])

In [ ]:
df.columns

In [ ]:
df, _ = op.motif(
    input_data=df,
    oligo_length_limit=170,
    motif_sequence_constraint='GACGTC'+'NNN', # The AatII motif with right flanking Ns
    maximum_repeat_length=6,
    motif_column='AatIIPadded',
    left_context_column='YellowReversePrimer',
    right_context_column=None,
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'])  # Exclude EcoRI, AatII, BsaI motifs

Step 3 in the `motif` pipeline warned us about the presence of `'GACGTC'` in our constraint itself. Nevertheless, the design step was successful.

In [ ]:
length_stats = op.lenstat(input_data=df, oligo_length_limit=170)

We can now use the `spacer` module to pad the oligos to 170 bases. Free space available is between 0 to 14 bases.

In [ ]:
? op.spacer

`Oligopool Calculator` can flexibly design our variable length spacers. Here our aim is to add a spacer so that the final oligo length is 170 bases, and we can achieve this by simply specifying the `oligo_length_limit` and using `None` as our `spacer_length` argument. Otherwise, we could specify the length of each spacer, in order, per oligo in a list or a DataFrame.

In [ ]:
df, _ = op.spacer(
    input_data=df,
    oligo_length_limit=170,
    spacer_length=None,    # Using None, we will automatically add spacers to match oligo_length_limit
    maximum_repeat_length=6,
    spacer_column='Spacer',
    left_context_column='AatIIPadded',
    right_context_column=None,
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'],  # Exclude EcoRI, AatII, BsaI motifs
    verbose=True)

In [ ]:
length_stats = op.lenstat(input_data=df, oligo_length_limit=170)

Our library is complete, all elements have been added, and the oligos are all 170 bases. Let's finalize our library using the `final` module.

In [ ]:
# ? op.final

In [ ]:
final_df, _ = op.final(input_data=df)

In [ ]:
final_df.head(5)

> **Note** All annotations of the individual oligos are lost in this step. So, we saved it to a new DataFrame. In general, it is a good idea to store the result of each step in `Design Mode` to separate DataFrame variables if we were designing our pools interactively, and wanted to roll back.

Using the `final` module produces a DataFrame that needs to be passed along for synthesis. `Design Mode` functions should no longer be applied on it.

Before shipping the library for synthesis, it is useful to run a quick QC pass. Compared to `lenstat`, the `verify` module is broader: it summarizes the architecture and length distribution, inspects column types (sequence vs metadata), and checks that excluded motifs did not re-emerge beyond their intended occurrence.

It also flags degenerate/IUPAC sequence columns (e.g., UMIs) so you can confirm they are intentional.

In [ ]:
? op.verify

In [ ]:
verify_stats = op.verify(
    input_data=final_df,
    oligo_length_limit=170,
    excluded_motifs=['GAATTC', 'GACGTC', 'GGTCTC', 'GAGACC'],  # Exclude EcoRI, AatII, BsaI motifs
    verbose=False)
verify_stats['status']

We can also explicitly check for any violation of our motif constraints. For example, in all our designed oligos there should only be single occurrences of the `EcoRI` and `AatII` motifs at their respective location, and no occurrences of the `BsaI` motif (since it will be used for pad excision later).

In [ ]:
assert final_df.CompleteOligo.transform(lambda x: x.count('GAATTC')).values.max() == 1

In [ ]:
assert final_df.CompleteOligo.transform(lambda x: x.count('GACGTC')).values.max() == 1
assert final_df.CompleteOligo.transform(lambda x: x.count('GGTCTC')).values.max() == 0  # No BsaI sites
assert final_df.CompleteOligo.transform(lambda x: x.count('GAGACC')).values.max() == 0  # No BsaI RC sites

This concludes our `Oligopool Calculator` `Design Mode` walkthrough.

### Assembly Mode Walkthrough

#### The Challenge: Synthesis Length Limits

Modern oligo synthesis has practical length limits - typically 150-300 bp depending on provider and chemistry. But many experimental designs require longer constructs:

**Common scenarios requiring assembly:**
- Multi-domain protein libraries (promoter + RBS + CDS + terminator)
- Long regulatory elements (enhancers spanning 500+ bp)
- Combinatorial screening with multiple functional units
- CRISPR arrays with multiple guide RNAs

**The solution:** Fragment your design into overlapping pieces that can be synthesized separately, then assembled via overlap-extension PCR (or similar overlap-based assembly). `Assembly Mode` provides two functions:
- **`split`**: Fragments long oligos into overlapping pieces with optimized assembly junctions
- **`pad`**: Adds Type IIS restriction sites for scarless excision after amplification

#### Splitting long oligos into overlapping fragments

The `split` function fragments long oligos into overlapping pieces optimized for assembly:
- Overlaps are designed with compatible melting temperatures for PCR assembly
- Hamming distance constraints prevent cross-assembly between fragments from different oligos
- Fragments are returned in alternating strand orientations (ready for PCR)

Treat each `SplitN` column as a separate synthesis pool: run `pad` once per split column, then `final` each padded DataFrame. If you prefer one DataFrame per fragment, enable `separate_outputs` (CLI `op split` writes separate files by default).

![Splitting and Padding](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/assembly.svg)

In [ ]:
? op.split

We will pretend that our 170 bp oligos from above are slightly longer than can be synthesized successfully in a pool where each oligo can be at most 150 bp long.

Our goal then is to fragment the oligos so that each fragment is at most 110 bases with 40 to 60 base overlap across the fragments. We will leave 40 bases, 20 bp each for the two terminal padding primers.

In [ ]:
split_df, _ = op.split(
    input_data=df,          # Notice, we are not using the 'final_df'
    split_length_limit=110, # Each split must be at most 110 bases long
    minimum_melting_temperature=55, # Assembly Tm is 55C
    minimum_hamming_distance=5,     # Ensure the overlaps are distinct
    minimum_overlap_length=40,    # At least 40 bp overlap
    maximum_overlap_length=60)    # At most 60 bp overlap

In [ ]:
split_df.head(5)

In [ ]:
length_stats = op.lenstat(input_data=split_df, oligo_length_limit=150);

We have automatically split our library for two-fragment assembly. The first pool contains 110 bp oligos while the oligos in the second pool are shorter.

The split fragments are in PCR assembly order, meaning the first fragment and the second fragments are returned in opposite strands, so that PCR generates the correct full-length double-stranded oligos. We can verify this using the `revcomp` module to change the orientation of `'Split2'` column and see that the ends of `'Split1'` sequences overlap with the beginning of the sequences in `'Split2'` column.

**Tip (not demoed):** Use `merge` to collapse a column range into a single DNA element before other operations.

This is helpful when a multi-column region (e.g., `ForwardPrimer` + `BC1` + `Variant` + `BC2`) should be treated as one contiguous sequence for downstream steps like `revcomp` or `final`.

Learn more: run `help(op.merge)` (or `? op.merge`) in this notebook.

In [ ]:
# help(op.revcomp)

In [ ]:
check_df, _ = op.revcomp(
    input_data=split_df,
    left_context_column='Split2',
    right_context_column='Split2',
    verbose=False)

In [ ]:
check_df.head(1)

In [ ]:
split1 = check_df.head(1).Split1.values[0]
split2 = check_df.head(1).Split2.values[0]

In [ ]:
for i in range(max(len(split1), len(split2)), 0, -1):
    if split1[-i:] == split2[:i]:
        alignment_padding = len(split1) - i
        print(split1)
        print(' ' * alignment_padding + '|' * i)
        print(' ' * alignment_padding + split2)
        break
else:
    print('No overlap between computed splits!')

# Verify overlap exists
assert any(split1[-i:] == split2[:i] for i in range(max(len(split1), len(split2)), 0, -1)), \
    'Split fragments should have valid overlap for assembly'

#### Padding fragments for Type IIS assembly

The `pad` module allows designing of padded primers with embedded Type IIS restriction sites to amplify the oligo fragments, convert them to double-stranded DNA (assuming synthesized oligos are ssDNA) and enable scarless assembly downstream.

In [ ]:
? op.pad

We will design our pads with `'BsaI'`, as an example. Note that we excluded the BsaI recognition motif (`GGTCTC` and its reverse complement `GAGACC`) from all earlier design elements (primers, barcodes, motifs, spacers) to ensure the Type IIS enzyme only cuts at the designed pad excision sites.

In [ ]:
first_pad_df, _ = op.pad(
    input_data=split_df,
    oligo_length_limit=150, # Recall, we are pretending 150 bp synthesis limit
    split_column='Split1',  # We will pad the first fragment
    typeIIS_system='BsaI',  # Pad with internal 3' BsaI motif
    minimum_melting_temperature=53,
    maximum_melting_temperature=55,
    maximum_repeat_length=10)

In [ ]:
op.lenstat(input_data=first_pad_df, oligo_length_limit=150);

Looks good. On to the second padding.

In [ ]:
second_pad_df, _ = op.pad(
    input_data=split_df,
    oligo_length_limit=150,
    split_column='Split2',  # We will pad the second fragment now
    typeIIS_system='BsaI',
    minimum_melting_temperature=53,
    maximum_melting_temperature=55,
    maximum_repeat_length=10,
    verbose=False)  # Silent Mode

In [ ]:
second_pad_df.head(5)

In [ ]:
op.lenstat(input_data=second_pad_df, oligo_length_limit=150);

This concludes our `Oligopool Calculator` `Assembly Mode` walkthrough.

**Key takeaways:**
- `split` fragments long oligos into overlapping pieces with Tm-optimized junctions
- Overlaps are Hamming-separated to prevent cross-assembly between different oligos
- `pad` adds Type IIS restriction sites for scarless excision after PCR amplification
- Run `pad` and `final` separately for each fragment pool before ordering synthesis

### Degenerate Mode Walkthrough

#### The Challenge: Expensive Similar-Sequence Libraries

Modern protein engineering increasingly relies on machine learning to predict optimal sequences. A typical workflow generates thousands of candidate variants - often highly similar sequences clustered around a predicted optimum.

**The cost problem:** Ordering 10,000 concrete oligos at \$0.10/base x 150 bp = **\$150,000** for synthesis alone. If those variants compress to 500 degenerate oligos, synthesis cost drops to **\$7,500** - a 20x reduction.

**The experiment design insight:** For many experiments (growth selection, FACS sorting, survival screens), you don't need to *count* every variant. Selection naturally enriches winners. You only need to *identify* which sequences survive.

`Degenerate Mode` addresses this by compressing similar concrete sequences into IUPAC-degenerate oligos. The compression is **lossless** - expanding the degenerate oligos recovers exactly your original variant set, with no invented sequences.

**Note:** Barcoded libraries typically compress poorly (barcodes are intentionally diverse), so Degenerate Mode is mainly used for selection assays where you identify winners by sequencing.

#### Analysis Mode vs Degenerate Mode

| | **Analysis Mode** | **Degenerate Mode** |
|---|---|---|
| **Goal** | Quantify all variants | Find top performers |
| **Readout** | Barcode abundance | Survivor identity |
| **Synthesis format** | Concrete oligos (one per variant) | IUPAC-degenerate oligos |
| **Cost scaling** | Linear with variants | Sublinear (compression) |
| **Analysis pipeline** | `index` -> `pack` -> `acount`/`xcount` | Sequence survivors + align |
| **Best for** | MPRA, expression maps, dose-response | Optimization, directed evolution, screening |

**When to use Degenerate Mode:**
- Libraries with similar sequences (good compression potential)
- Directed evolution or optimization screens
- Growth-based, FACS-based, or survival selection assays
- Cost-sensitive experiments with large variant pools
- Experiments where you can identify winners by sequencing (no barcode counting)

**Stick with Analysis Mode when:**
- You need quantitative abundance measurements for every variant
- Variants are highly dissimilar (poor compression, minimal savings)
- You need barcode-based tracking through complex workflows

#### The Degenerate Mode Pipeline

The Degenerate Mode workflow integrates seamlessly with Design Mode. Start by designing your library with any Design Mode methods as required and `verify` all constraints, then replace the final `final` step with `compress`. This produces a smaller set of IUPAC-degenerate oligos that encode all your variants.

After synthesis, amplify and clone the degenerate pool, then run your selection assay (growth, FACS, survival screen). The selection naturally enriches winners. Finally, sequence the survivors and use `mapping_df` to trace each surviving sequence back to its original variant ID(s).

The key insight: you don't need barcodes or counting infrastructure. Selection does the work, and `mapping_df` provides traceability.

#### Understanding IUPAC Degenerate Codes

IUPAC codes extend the standard A/T/G/C alphabet to represent positions that can be any subset of bases. Synthesis vendors interpret these codes and produce a mixture of oligos covering all combinations.

| Code | Name | Bases | Degeneracy |
|:---:|:---|:---:|:---:|
| A | Adenine | A | 1 |
| C | Cytosine | C | 1 |
| G | Guanine | G | 1 |
| T | Thymine | T | 1 |
| R | Purine | A, G | 2 |
| Y | Pyrimidine | C, T | 2 |
| S | Strong | G, C | 2 |
| W | Weak | A, T | 2 |
| K | Keto | G, T | 2 |
| M | Amino | A, C | 2 |
| B | Not A | C, G, T | 3 |
| D | Not C | A, G, T | 3 |
| H | Not G | A, C, T | 3 |
| V | Not T | A, C, G | 3 |
| N | Any | A, C, G, T | 4 |

**Example:** The sequence `ATGCNATGC` has degeneracy 4 (one `N` position x 4 bases). It represents the set: `{ATGCAATGC, ATGCCATGC, ATGCGATGC, ATGCTATGC}`.

The total degeneracy of a sequence is the product of degeneracies at each position. A sequence with two `N` positions has degeneracy 4 x 4 = 16.

In [ ]:
# Oligopool Calculator stores IUPAC codes internally as ddna_space
# (degenerate DNA space) - a mapping from code to the set of bases it represents
from oligopool.base.utils import ddna_space

# Show the mapping (excluding '-' placeholder)
for code, bases in ddna_space.items():
    if code != '-':
        print(f'{code}: {sorted(bases)} (degeneracy {len(bases)})')

#### Compressing a library into degenerate oligos

Let's see how `compress` works with a simple example first, then apply it to a more realistic ML-style library.

In [ ]:
? op.compress

**Toy Example: 12 variants -> ? degenerate oligos**

Let's create a small library where some sequences differ only at a few positions. This is representative of libraries with low mutational diversity where variants cluster around similar sequences.

In [ ]:
# 12 variants: two clusters of similar sequences
# Cluster 1: ATGCATGC... with variations at positions 9, 10, 11
# Cluster 2: GCTAGCTA... with variations at positions 10, 11, 12
toy_df = pd.DataFrame({
    'ID': [f'V{i:03d}' for i in range(1, 13)],
    'Sequence': [
        # Cluster 1: positions 9-12 vary (last 4 bases)
        'ATGCATGCATGC',  # V001
        'ATGCATGCATGT',  # V002
        'ATGCATGCATGA',  # V003
        'ATGCATGCATGG',  # V004
        'ATGCATGCCTGC',  # V005: also varies at position 9
        'ATGCATGCGTGC',  # V006
        'ATGCATGCTTGC',  # V007
        'ATGCATGCCCGC',  # V008
        # Cluster 2: different prefix, varies at last 2-3 bases
        'GCTAGCTAGCTA',  # V009
        'GCTAGCTAGCTT',  # V010
        'GCTAGCTAGCTG',  # V011
        'GCTAGCTAGCAA',  # V012: 2 positions different
    ],
})
print(f'Input: {len(toy_df)} concrete variants')
toy_df

In [ ]:
# Compress the toy library
# rollout_simulations and rollout_horizon control the Monte Carlo search
# Higher values = better compression but slower runtime
toy_mapping_df, toy_synthesis_df, toy_stats = op.compress(
    input_data=toy_df,
    rollout_simulations=20,   # Monte Carlo rollouts per decision
    rollout_horizon=3,        # Lookahead positions
    random_seed=42,           # For reproducibility
)

`compress` returns **three outputs**:

1. **`mapping_df`**: Maps each original variant ID to its concrete sequence and the degenerate oligo ID that covers it. This is your traceability map for post-selection analysis.

2. **`synthesis_df`**: The degenerate oligos to order from your synthesis vendor. Each row is one oligo with its IUPAC sequence, degeneracy (how many concrete sequences it represents), and length.

3. **`stats`**: Compression statistics including compression ratio, degeneracy distribution, etc.

In [ ]:
# mapping_df: your lookup table for post-selection identification
# Each variant ID maps to its concrete sequence and degenerate oligo ID
print('Mapping DataFrame (variant -> degenerate oligo):')
display(toy_mapping_df)

In [ ]:
# synthesis_df: what you actually order from the vendor
# Each degenerate oligo encodes multiple concrete variants
print('Synthesis DataFrame (order sheet):')
display(toy_synthesis_df)

In [ ]:
print(f'\nCompression: {len(toy_df)} variants -> {len(toy_synthesis_df)} degenerate oligos '
      f'({len(toy_df)/len(toy_synthesis_df):.1f}x compression)')

In [ ]:
# Compression statistics
pprint(toy_stats)

#### Realistic Example: 500 ML-style variants

Now let's simulate a more realistic scenario: an ML model has predicted 500 variants clustered around 5 template sequences, each with 0-3 point mutations. This is typical of protein optimization workflows.

In [ ]:
ml_rng = np.random.default_rng(seed=123)  # Reproducible

In [ ]:
# 5 template sequences (40 bp each) - representing ML-predicted optimal regions
templates = [
    'TTGACAATTAATCATCGAACTAGTTAACTAGTACGCAAG',
    'CGTACGATCGATCGATCGATCGATCGATCGATCGATCGA',
    'ATGCATGCATGCATGCATGCATGCATGCATGCATGCATGC',
    'GCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA',
    'AACCGGTTAACCGGTTAACCGGTTAACCGGTTAACCGGTT',
]

In [ ]:
def mutate_sequence(seq, n_mutations, rng):
    '''Introduce n random point mutations into sequence.'''
    seq = list(seq)
    positions = rng.choice(len(seq), size=n_mutations, replace=False)
    for pos in positions:
        bases = ['A', 'T', 'G', 'C']
        bases.remove(seq[pos])  # Don't substitute with same base
        seq[pos] = rng.choice(bases)
    return ''.join(seq)

In [ ]:
# Generate variants: 100 per template, 0-3 mutations each.
ml_variants = []
for template_idx, template in enumerate(templates):
    for i in range(100):
        n_mutations = ml_rng.integers(0, 4)  # 0, 1, 2, or 3 mutations
        variant = mutate_sequence(template, n_mutations, ml_rng)
        ml_variants.append({
            'ID': f'ML_{template_idx+1:02d}_{i+1:03d}',
            'Sequence': variant,
            'Template': f'T{template_idx+1}',
            'Mutations': n_mutations
        })

In [ ]:
ml_df = pd.DataFrame(ml_variants)
print(f'Generated {len(ml_df)} ML-style variants from {len(templates)} templates')
print(f'Mutation distribution: {ml_df.Mutations.value_counts().sort_index().to_dict()}')
ml_df.head(10)

In [ ]:
# Compress the ML library (using only ID and Sequence columns)
ml_compress_input = ml_df[['ID', 'Sequence']].copy()

ml_mapping_df, ml_synthesis_df, ml_stats = op.compress(
    input_data=ml_compress_input,
    rollout_simulations=50,   # More simulations for better compression
    rollout_horizon=4,        # Deeper lookahead
    random_seed=42,
    verbose=False,            # Silent mode for cleaner output
)

In [ ]:
# Compression results
n_variants = len(ml_df)
n_degenerate = len(ml_synthesis_df)
compression_ratio = n_variants / n_degenerate

print(f'Compression Results:')
print(f'     Input variants: {n_variants:,}')
print(f'  Degenerate oligos: {n_degenerate:,}')
print(f'  Compression ratio: {compression_ratio:.1f}x')

In [ ]:
print(f'Cost savings estimate (at $0.10/base x 40 bp):')
print(f'  Concrete synthesis:   ${n_variants * 40 * 0.10:,.0f}')
print(f'  Degenerate synthesis: ${n_degenerate * 40 * 0.10:,.0f}')
print(f'  Savings:              ${(n_variants - n_degenerate) * 40 * 0.10:,.0f}')

#### Expanding degenerate oligos

The `expand` function reverses compression: it takes synthesis_df (degenerate oligos) and returns all concrete sequences they encode. This is useful for:
- **Verification**: Confirm that expanding the compressed output recovers exactly the set of unique input sequences (lossless guarantee)
- **Reference building**: Generate the full concrete sequence set for alignment

In [ ]:
? op.expand

In [ ]:
# Verify lossless guarantee: expand(compress(X)) == X
# Expand the toy example and check that we get back the original sequences
toy_expanded_df, toy_expand_stats = op.expand(
    input_data=toy_synthesis_df,
    sequence_column='DegenerateSeq',
    verbose=False,
)
toy_expanded_df

In [ ]:
# The expanded sequences should exactly match our original input
original_sequences = set(toy_df['Sequence'].values)
expanded_sequences = set(toy_expanded_df['ExpandedSeq'].values)

In [ ]:
print(f'Original sequences:  {len(original_sequences)}')
print(f'Expanded sequences:  {len(expanded_sequences)}')
print(f'Match: {original_sequences == expanded_sequences}')

In [ ]:
# Assertion to verify lossless compression
assert original_sequences == expanded_sequences, 'Lossless guarantee violated!'
print('\nLossless guarantee verified for toy example.')

This concludes our `Oligopool Calculator` `Degenerate Mode` walkthrough.

**Key takeaways:**
- `compress` reduces synthesis cost by encoding similar sequences in IUPAC-degenerate oligos
- The compression is **lossless**: expanding `synthesis_df` recovers exactly the set of unique input sequences
- Use `mapping_df` to trace survivors back to original variant IDs after selection
- File outputs use distinct suffixes: `mapping_file` -> `.oligopool.compress.mapping.csv`, `synthesis_file` -> `.oligopool.compress.synthesis.csv`
- Best for selection assays: sequence survivors and map winners back to variant IDs

### Analysis Mode Walkthrough

#### The Challenge: Extracting Signal from Sequencing Noise

After synthesizing and characterizing your oligo pool library, you need to quantify variant activity from NGS reads. This sounds simple - just count barcodes - but real-world data presents challenges:

**The problem:**
- Millions of reads with sequencing errors
- Barcodes may be partially obscured by errors
- Need to verify barcode-variant associations survived cloning
- Multiple experimental designs require different counting strategies

**The solution:** `Analysis Mode` provides a specialized pipeline optimized for barcoded oligo pool quantification:
- **`index`**: Train a `Scry` classifier on your barcode+variant architecture
- **`pack`**: QC-filter and deduplicate reads for efficient counting
- **`acount`/`xcount`**: Count with error-tolerant matching and optional callbacks

![Ribozyme Library Architecture](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/architecture-ribozyme.svg)

In our `Design Mode` walkthrough we designed a pool of over 6,000 simulated ribozyme variants. One characterization approach may involve sequencing an amplicon spanning `OrangeForwardPrimer` to `YellowReversePrimer` for uncleaved products, while a second sequencing of cleaved products spans `PinkForwardPrimer` through `YellowReversePrimer`. The ratio of cleaved to uncleaved products would give us the cleavage rate quantification for a ribozyme variant.

`Oligopool Calculator` `Analysis Mode` provides efficient tools to analyze barcoded read counts to facilitate this step, in the form of `index`, `pack`, `acount` and `xcount` modules. In this section we will explore them.

#### Indexing the barcodes and associated variants

We first have to `index` the barcodes and associated variants to train a `Scry` barcode classifier for downstream counting. This step is analogous to indexing reference genomes that short-read aligners require before mapping NGS reads for genomic analysis, except our `Analysis Mode` functions are optimized for counting and therefore very sensitive and efficient for this task.

**Anchors and multi-anchor reads:** `index` can use constant prefix/suffix anchor columns to localize barcodes (and optional associates) in reads. If an anchor appears multiple times in a read, the counting engine uses the best-scoring placement; ties (ambiguous anchors) are rejected. This is why anchor design (typically via `motif(motif_type=1, ...)` and repeat screening) matters.

![Indexing Reads](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/index.svg)

In [ ]:
? op.index

Althrough not explored here, you can learn more about using `Scry` via `help(op.Scry)`.

For demonstration, we will load an annotated oligo pool we generated in the previous section from which we simulated a naive ribozyme cleavage experiment. The notebooks, reference CSV, and the simulated NGS reads are all located in our `analysis-pipeline` directory inside `examples`. The simulation parameters, in particular, can be adjusted in `read_simulator.ipynb` to test `Analysis Mode` functions with different characterization profiles.

In [ ]:
df = pd.read_csv('analysis-pipeline/ribozyme_architecture.csv')

In [ ]:
df.head(5)

We can now index the two barcode sets, since indexing requires us to index them individually. For BC2 in particular, we will associate the core variants for association counting later.

In [ ]:
# For demo, remove any previous indexes
! rm -f BC*.oligopool.index

In [ ]:
stats = op.index(
    barcode_data=df,
    barcode_column='BC1',                        # Index BC1 only
    barcode_prefix_column='OrangeForwardPrimer', # constant prefix anchor adjacent to BC1
    barcode_suffix_column=None,                  # We have no flanking right constant for BC1
    barcode_prefix_gap=0,
    barcode_suffix_gap=0,
    index_file='BC1',                            # Store results in BC1.oligopool.index file
    verbose=False,       # Silent Mode
)

In [ ]:
stats = op.index(
    barcode_data=df,
    barcode_column='BC2',                        # Index BC2 only
    barcode_prefix_column='PinkForwardPrimer',   # constant prefix anchor adjacent to BC2
    barcode_suffix_column='YellowReversePrimer', # We have YellowReversePrimer as flanking right constant for BC2
    barcode_prefix_gap=0,
    barcode_suffix_gap=0,
    associate_data=df,                           # Associates are in same DataFrame
    associate_column='Variant',
    associate_prefix_column=None,
    associate_suffix_column='PinkForwardPrimer',
    associate_prefix_gap=0,
    associate_suffix_gap=0,
    index_file='BC2',                            # Store results in BC2.oligopool.index file
)

> **Note** Without specifying the associated variants for a barcode set, the resulting index cannot be used for association counting. Here, the `BC1` index can only be used for combinatorial counting, while `BC2` can be used for both association and combinatorial counting.

Our barcodes and their associates are indexed. Let us now pack the NGS reads for counting.

#### Packing the NGS reads for quantification

NGS analysis begins with quality control of reads. Only high quality reads should be used for characterization. The `pack` module allows us to filter low quality or incorrect reads so that our quantifcation downstream is accurate.

Optionally, if the reads are paired-end and long enough, compared to the synthesized oligo, the two mates may need to be merged into a consensus fragment, which can be done within the packing step. Alternatively, efficient tools such as [`flash2`](https://github.com/dstreett/FLASH2) can be used for the same -- merge your reads with `flash2` and use the resulting single-end reads in `pack`.

Finally, due to the nature of MPRA characterization, there may be an abundance of truly duplicate reads which `pack` module consolidates into read packs for efficient and parallel counting downstream.

In practice, you may wish to add [unique molecular indexes](https://www.illumina.com/techniques/sequencing/ngs-library-prep/multiplexing/unique-molecular-identifiers.html) (UMIs) during your library preparation to remove spurious PCR or optical duplicates. We recommend directly processing your reads with tools such as [`calib`](https://github.com/vpc-ccg/calib) to consolidate these UMI duplicates, prior to packing and counting them using `Oligopool Calculator`.

![Packing Reads](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/pack.svg)

In [ ]:
? op.pack

In [ ]:
# Remove previous pack, if it exists
! rm -f NGS.oligopool.pack

In [ ]:
stats = op.pack(
    r1_fastq_file='analysis-pipeline/ribozyme_1M_R1.fq.gz',
    r2_fastq_file='analysis-pipeline/ribozyme_1M_R2.fq.gz',
    r1_read_type='forward',     # R1 is in Forward Orientation
    r2_read_type='reverse',     # R2 is in Reverse Orientation
    minimum_r1_read_quality=30, # Filter out any reads with Phred Score less than 30
    minimum_r2_read_quality=30,
    minimum_r1_read_length=10,  # Filter out any reads shorter than 10 bases
    minimum_r2_read_length=10,
    pack_type='merge',          # R1 and R2 needs to be merged
    pack_size=0.1, # Store 100K reads per read pack
    pack_file='NGS',
    core_count=4,  # Use only 4 CPU cores
)

All our reads are packed and we are ready to count the barcodes. We can see that all reads "survived" our filters and were merged successfully. Despite sequencing a million reads, we only need to analyze less than 20 thousand reads in the end.

#### Overview of counting methods

The final step of the analysis pipeline utilizes packed reads and indexed barcode/variant information to generate a read count matrix, which is essential for tabulating variant activity levels. `Oligopool Calculator` `Analysis Mode` offers two efficient counting functions tailored to experimental needs: association counting and combinatorial counting. Users can define parameters for error tolerance and mapping strategy, and they have the option to specify custom callback functions that execute concurrently with the mapping and counting process.

![Counting Reads](https://raw.githubusercontent.com/ayaanhossain/repfmt/refs/heads/main/oligopool/img/count.svg)

#### `acount` vs `xcount`: Choosing Your Counting Method

| | **acount** (Association) | **xcount** (Combinatorial) |
|---|---|---|
| **Goal** | Verify barcode-variant pairing | Count barcode combinations |
| **Index requirement** | Single index with associates | One or more indexes |
| **Validation** | Checks variant sequence match | Maps barcodes independently |
| **Output** | `BarcodeCounts` + `AssociationCounts` | `CombinatorialCounts` |
| **Best for** | QC synthesis, input library | Activity assays, multi-barcode designs |
| **Reads used** | Only if barcode+variant match | If any barcode matches |

**Use `acount` when:**
- Verifying synthesis fidelity (did we get the designed barcode-variant pairs?)
- Quantifying input library distribution before selection
- Validating cloning did not scramble associations

**Use `xcount` when:**
- Counting combinations of multiple barcodes (e.g., cleaved vs uncleaved)
- Flexible designs where not all barcodes may be present in every read
- Activity assays where you track barcode ratios

#### Association counting of variants and barcodes

In association counting, the goal is to verify the presence of designed variants uniquely assigned to specific barcodes. This function is useful for confirming oligo pool synthesis as an initial check before cloning, or for quantifying input library distribution, or validating the association between barcodes and variants after modifications, such as replacing a spacer with a reporter protein and verifying the association across the terminal ends. The process involves mapping the core barcode after realigning the read with respect to associate prefix or suffix constants, and verifying the presence of the extracted variant subsequence against the full reference variant sequence. Reads are discarded if associate constants are missing or if the core variant has too many errors and thus cannot be verified.

In [ ]:
? op.acount

In [ ]:
# Remove previous AC output, if it exists
! rm -f AC.oligopool.acount.csv

Let's measure the association of our ribozyme pool based on BC2 index (BC1 index is not usable here).

In [ ]:
ac_df, _ = op.acount(
    index_file='BC2.oligopool.index', # Map BC2
    pack_file='NGS.oligopool.pack',   # in our reads
    count_file='AC',
    mapping_type='sensitive', # Use sensitive/slow classification
    barcode_errors=-1,   # Use model-based auto-inferred error tolerance
    associate_errors=-1,
    callback=None,
    core_count=0,
    memory_limit=0.0,
    verbose=True)

None of the reads were mis-associated, as in everytime we detected a barcode, its associated variant was also found to be present in the read within error tolerance. This is of course artificial in our simulated example.

In [ ]:
ac_df.head(5)

We have our counts! The `BarcodeCounts` column shows the number of times the barcode was detected in a read, while `AssociationCounts` shows the number of times the barcode was unambiguously present with its designated associated variant.

In this simulated case, all reads are error-free, so `AssociationCounts` is the same as `BarcodeCounts`. In real-life scenarios, we expect `AssociationCounts` to be <= `BarcodeCounts` because not all barcoded reads retain an intact associate.

Let's extract the ranks of the designs and make some plots.

In [ ]:
ac_df['variant_rank'] = ac_df['BC2.ID'].transform(lambda x: int(x.split('_')[-1]))
ac_df['variant_rank_bin'] = ac_df.variant_rank.transform(lambda x: x//1000)
ac_df['variant_norm_read_count'] = (ac_df.BarcodeCounts - ac_df.BarcodeCounts.mean()) / ac_df.BarcodeCounts.std()

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(16,7))
sns.scatterplot(
    data=ac_df,
    x='variant_rank', y='BarcodeCounts',
    hue='variant_rank_bin',
    palette='Paired',
    s=5, ec=None,
    ax=ax[0]).set(
        ylabel='read_count')
sns.boxplot(
    data=ac_df,
    x='variant_norm_read_count',
    fill=False, notch=True, linewidth=1,
    hue='variant_rank_bin',
    palette='Paired',
    ax=ax[1])
plt.show()

As simulated, the higher the rank of the variant, the more frequently it appeared. In real experimental data, if the pool being sequenced has not undergone selection (say input library) then we expect this distribution to be uniform across all variants.

#### Combinatorial barcode counting with callback

Combinatorial counting is employed when a single or multiple barcode combinations are expected within sequenced reads. `xcount` uses multiple indexes to map the barcodes independently, allowing for flexibility in experimental design. Reads are only discarded if none of the expected barcodes are present, meaning that one or more missing barcodes are tolerated. This approach can accommodate isolated barcodes with their own flanking constants or unique combinations of sub-sequences within a larger assembly through combinatorial methods.

In [ ]:
? op.xcount

Let's apply combinatorial counting using both BC1 and BC2 index here.

In [ ]:
# Remove previous CC output, if it exists
! rm -f CC.oligopool.xcount.csv

In [ ]:
cc_df, _ = op.xcount(
    index_files=['BC1', 'BC2'],     # Map combinations of BC1 and BC2
    pack_file='NGS.oligopool.pack',
    count_file='CC',
    mapping_type='sensitive',       # Use sensitive/slow classification
    barcode_errors=-1,
    callback=None,
    core_count=0,
    memory_limit=0.0,
    verbose=True)

In [ ]:
cc_df

As we can see, we have counts of either BC2 or both BC1 and BC2 barcodes from our reads, signifying cleaved and ucleaved products respectively. We can now use this count matrix to compute the activity of the ribozymes.

In [ ]:
cc_df['ID'] = cc_df['BC2.ID']
cc_df['count_basis'] = cc_df.apply(lambda x: 'cleaved_reads' if x['BC1.ID'] != x['BC2.ID'] else 'uncleaved_reads', axis=1)
act_df = cc_df.pivot(index='ID', columns='count_basis', values='CombinatorialCounts').fillna(0.).reset_index()
act_df['activity'] = act_df.apply(lambda x: 100. * x.cleaved_reads / (x.cleaved_reads + x.uncleaved_reads), axis=1)
act_df = act_df.merge(ac_df, left_on='ID', right_on='BC2.ID').drop(columns=['BC2.ID']).rename(columns={'BarcodeCounts': 'input_reads'})

In [ ]:
act_df.tail(5)

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(16,7))
sns.scatterplot(
    data=act_df,
    x='input_reads', y='activity',
    hue='variant_rank_bin',
    palette='Paired',
    s=5, ec=None,
    ax=ax[0])
sns.scatterplot(
    data=act_df,
    x='cleaved_reads', y='uncleaved_reads',
    hue='activity',
    palette='viridis',
    s=5, ec=None,
    ax=ax[1])
plt.show()

As simulated, the higher the rank of the variant the more input reads it had, which is proportional to the inferred activity. This is seen in the plot on the left. The right plot shows those high activity variants having a larger proportion of cleaved reads relative to their uncleaved read counts.

Finally, let us use a callback function to filter out any reads not matching our simulated criteria of the cleavage location being the 10th base of the variant.

In [ ]:
df.head(5)

To do this we will observe a couple of things. First, the cleaved reads will only contain `BC2` in them. Second, the subsequence specified in the `Variant` column will appear partially in the read. Infact as simulated, the cleaved reads will contain part of the variant, the whole of `PinkForwardPrimer` and `BC2`.

Both `xcount` and `acount` allows us to use the information from the reads to concurrently do our own calculations on the reads and to signal to the counting algorithm whether to use the read as part of the counting process.

In [ ]:
# Remove previous CC output, if it exists
! rm -f CCC.oligopool.xcount.csv CCC.oligopool.xcount.failed_reads.csv

In [ ]:
# Build a shared dictionary for our callback below (detailed later)
ribozyme_memory = Manager().dict(zip(df.ID, df.Variant))

In [ ]:
# Callback logic
def filter_non_cleaved_reads(ribozyme_memory, pink_forward_primer):
    '''Outer wrapper to give our method some memory.'''

    def wrapped(r1, r2, ID, count, coreid):
        '''Inner closure to filter reads.

        Note that `xcount` and `acount` callbacks must have this signature (r2 is None for merged reads).
        See help(op.acount) or help(op.xcount) for details.'''

        # If more than one barcode present, we are not going to do anything with it, because
        # it implies an uncleaved read. We only want to use BC2-exclusive cleaved reads.
        if ID[0] is None and not ID[1] is None:
            # The first ID will be None during `xcount` because we will compute BC1 x BC2

            # Extract ribozyme ID
            ribozyme_id = ID[1]

            # The constant primer must be present for acceptance
            reads = [r1] if r2 is None else [r1, r2]
            for read in reads:
                if pink_forward_primer in read:

                    # Extract ribozyme fragment
                    ribozyme_fragment = read[:read.index(pink_forward_primer)]
                    # Fetch the expected full variant
                    ribozyme_full = ribozyme_memory[ribozyme_id]

                    # Figure out where the fragment starts in the variant
                    if ribozyme_fragment in ribozyme_full:
                        ribozyme_start = ribozyme_full.index(ribozyme_fragment)
                        # This index must be 10 or greater for cleaved product-based read
                        return ribozyme_start >= 10

            '''
            Note: Here we are using exact matching logic. In reality we might want to do
                  some alignment based extraction of our fragments to be error-aware.
            '''

        # Reject the read for counting ... didn't pass filtering due to one or more reasons
        return False

    # Return Wrapped Processor
    return wrapped

Virtually any logic can be executed concurrently with counting, as long as the callback returns a boolean. If a callback function returns `False`, the counting function will not use the reads for counting.

If we need to, we can also record additional results from our callback calculations via [shared data structures](https://superfastpython.com/multiprocessing-manager-example/) such as a `Manager` `dict` or `list`. This is useful, for example, if we wanted to extract and store the location of cleavage sites in reads (instead of assuming it to be the 10th base in the designed variants). Another similar use case is extracting transcription start sites for a library of designed promoters.

> **Note** The `Manager` `dict` here ensures that all parallel callback processes during counting will have access to the same exact dictionary, controlled and synchronized by a separate `Manager` instance. This allows us to modify the content of the managed dictionary in one process and have the changes be reflected in callbacks executed by other counting processes in parallel. This also allows us to easily gather and store additional results, in parallel, to a persistent and centralized dictionary for post-counting analysis.

In [ ]:
# Notice that we need to use the outer wrapper as callback
cc_callback = filter_non_cleaved_reads(
    ribozyme_memory=ribozyme_memory,
    pink_forward_primer=df.PinkForwardPrimer.values[0])

Don't worry, if our callback fails, the return stats dictionary will contain the inputs for which our callback failed and we can debug accordingly. If you'd like to see *examples* of discarded reads (including failures from callbacks, barcode detection, anchoring, low complexity, and PhiX), we can enable `failed_reads_file` to write a small per-category sample to disk, as we do below.

In [ ]:
_, stats = op.xcount(
    index_files=['BC1', 'BC2'],
    pack_file='NGS.oligopool.pack',
    count_file='CCC',
    failed_reads_file='CCC',            # Sample failed reads for diagnostics
    failed_reads_sample_size=500,       # Max samples per failure category
    mapping_type='sensitive',           # Use sensitive/slow classification
    barcode_errors=-1,
    callback=cc_callback,
    core_count=0,
    memory_limit=0.0,
    verbose=False) # Silent Mode

In [ ]:
pprint(stats)

We can see that our callback filtering function rejected a lot of reads, as recorded in `callback_false_reads` key in the stats dictionary. Rest of the reads were cleaved and were accepted for counting.

We can now investigate our `failed_reads_file`.

In [ ]:
fail_df = pd.read_csv('CCC.oligopool.xcount.failed_reads.csv')
fail_df['failure_reason'].value_counts()

In [ ]:
fail_df.head(10)

This concludes our `Oligopool Calculator` `Analysis Mode` walkthrough.

**Key takeaways:**
- `index` trains a `Scry` classifier on your barcode architecture (include associates for `acount`)
- `pack` QC-filters reads and deduplicates them into efficient read packs
- Use `acount` to verify barcode-variant associations (synthesis QC, input library distribution)
- Use `xcount` to count barcode combinations (activity assays, multi-barcode designs)
- Custom callbacks enable concurrent read filtering/processing during counting
- Use `Manager` dicts for thread-safe shared state across parallel callbacks

In [ ]:
! rm -f BC*.oligopool.index NGS.oligopool.pack AC.oligopool.acount.csv CC*.oligopool.xcount.csv CCC.oligopool.xcount.failed_reads.csv

### Conclusion

`Oligopool Calculator` offers a comprehensive solution for building and analyzing genetic part libraries using oligo pools. `Design Mode` helps you compose constrained, synthesis-ready architectures; `Assembly Mode` supports post-synthesis fragment assembly for long constructs; `Degenerate Mode` enables cost-efficient selection workflows via IUPAC-degenerate synthesis; and `Analysis Mode` provides fast, flexible barcode-based quantification from NGS reads. We hope it helps you spend less time reinventing pipelines and more time learning biology.

---